In [1]:
from sentence_transformers import SentenceTransformer, util

/home/davor/moj-chatbot-search/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import DirectoryLoader

In [3]:
loader = DirectoryLoader("data/documents/", glob="*.pdf")

In [4]:
documents = loader.load()
# DOWNLOADS TO /home/davor/nltk_data

In [29]:
documents

[Document(page_content='ePredmet\n\n1. Zašto ne mogu pronaći predmet? (cid:0)\n\nZbog sigurnosnih razloga, u ovoj bazi podataka nije moguće pronaći kaznene i prekršajne maloljetničke predmete, predmete istrage i ratnih zločina te predmete iz nadležnosti USKOK-a.\n\n2. Zašto ne mogu pronaći Z predmet? (cid:0)\n\nZemljišnoknjižni predmeti nisu dostupni u ovoj bazi podataka, jer se vode u zasebnom sustavu te se oni mogu pratiti na poveznici https://oss.uredjenazemlja.hr/.\n\n3. (cid:0)\n\nInformacije o predmetu ePredmet je javna i besplatna usluga strankama, punomoćnicima i drugim zainteresiranim osobama koje sudjeluju u sudskim postupcima, a obuhvaća predmete općinskih, županijskih, upravnih, trgovačkih sudova, Visokog prekršajnog suda, Visokog trgovačkog suda, Visokog upravnog suda, Visokog kaznenog suda i Vrhovnog suda RH.\n\nNa Internet stranici http://e-predmet.pravosudje.hr/?q=pretraga biranjem nadležnog suda sa padajućeg izbornika i upisom točnog broja predmeta, možete vidjeti stat

In [5]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [6]:
splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=50,
    model_name="./data/model_data/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2"
    # model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
)

In [7]:
split_documents = splitter.split_documents(documents)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name="./data/model_data/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [37]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(split_documents, embeddings_model, persist_directory='.data/vectorstore.db')
retriever = db.as_retriever()

In [38]:
retriever.invoke(
    """kako da nađem sudski predmet?"""
)

[Document(page_content='ćenici, ako ih imaju, odabir om opcije. Prilikom unosa zastupnika odnosno opunomoćenika, potrebno je, osim u nosa osnovnih podataka o osobi, unijeti i podatak o vrsti zastupanja, k ojeg sudionika zastupa te pravnu osnovu za zastupanj e. Ako je potrebno priložiti punomoć odnosno dokaz o zastupanju, to se može učiniti klikom na:', metadata={'doc_id': 231, 'doc_real_id': 6, 'page': 9, 'source': 'local'}),
 Document(page_content='Potvrda slanja izgleda ovako: Na modulu Poslani na sud postoji opcija pretraživanja (filtriranja) podataka. Ova forma je "skrivena" kako ne bi zauzimala nepotreban prostor na ekranu ako se ne koristi. Ista se otvara klikom na strelicu prema dolje.', metadata={'doc_id': 250, 'doc_real_id': 6, 'page': 28, 'source': 'local'}),
 Document(page_content='šenika za dug razmjerna ili solidarna, pri čemu je razmjerna odgovornost unaprij ed zadana vrijednost. Na prozoru u nosa podataka o ovršeni ku unose se i njihovi zastupnici odnosno opunomoćenici, 

In [17]:
embeddings = embeddings_model.embed_documents([split_document.page_content for split_document in split_documents])

In [21]:
len(embeddings), len(embeddings[0])

(133, 384)

In [2]:
model = SentenceTransformer(
    "./data/model_data/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2",  # "paraphrase-multilingual-MiniLM-L12-v2",
)

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [12]:
import psycopg2

In [27]:
# Open a cursor to perform database operations
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="TWbyVdR1rt%+",
    host="3.75.212.152",
    port="5432",
)
cur = conn.cursor()
cur.execute(
    f"SELECT * FROM pages"
)
examples = cur.fetchall()
cur.close()
conn.close()

In [30]:
from langchain.docstore.document import Document

documents = []
for example in examples:
    doc = Document(page_content=example[1], metadata={"source": 'local', "doc_id": example[0], 'doc_real_id': example[2], 'page': example[3], })
    documents.append(doc)

In [31]:
split_documents = splitter.split_documents(documents)

In [34]:
split_documents

[Document(page_content="Na 'Ukupni iznos'kod oznake unosi se iznos tražbine u kunama (glavnice i kamate), što korisnik sam računa i unosi. Obvezno je unijeti podatke o ukupnoj glavnici i ukupnom iznosu. Podaci se spremaju korištenjem opcije pored svake tabele. Primjer 5. isplata u stranoj banci Opciju kod pod ataka za isplatu prebacite na 'NE' (Isplata u domaću banku)", metadata={'source': 'local', 'doc_id': 285, 'doc_real_id': 6, 'page': 63}),
 Document(page_content='Otvara se polje s podacima o stranoj banci odnosno vlasniku računa u toj banci koje je potrebno popuniti.', metadata={'source': 'local', 'doc_id': 286, 'doc_real_id': 6, 'page': 64}),
 Document(page_content='11. Primjeri unosa podataka u obrazac – najčešća pitanja 11.1 Unos suvlasnika stambene zgrade Primjer 1. Unos suvlasnika stambene zgrade kao ovrhovoditelja/ovršenika Kada se unose suvlasnici stambene zgrade kao ovrhovoditelj/ovršenik bira se kategorija Ostalo te se unose potrebni podaci uključujući Račun specifične na

In [35]:
db = Chroma.from_documents(split_documents, embeddings_model)
retriever = db.as_retriever()

In [36]:
retriever.invoke("""kako da nađem sudski predmet?""")

[Document(page_content='uju u sudskim postupcima, a obuhvaća predmete općinskih, županijskih, upravnih, trgovačkih sudova, Visokog prekršajnog suda, Visokog trgovačkog suda, Visokog upravnog suda, Visokog kaznenog suda i Vrhovnog suda RH. Na Internet stranici http://e-predmet.pravosudje.hr/?q=pretraga biranjem nadležnog suda sa padajućeg izbornika i upisom točnog broja predmeta, možete vidjeti status vašeg predmeta. Ministarstvo pravosuđa i uprave kao najviše tijelo za obavljanje pravosudne', metadata={'source': 'data/documents/Popis pitanja za chatbot.docx.pdf'}),
 Document(page_content='hr/?q=pretraga biranjem nadležnog suda sa padajućeg izbornika i upisom točnog broja predmeta, možete vidjeti status vašeg predmeta. Ministarstvo pravosuđa i uprave kao najviše tijelo za obavljanje pravosudne uprave osigurava samo tehničke preduvjete za javnu i besplatnu uslugu ePredmet. Svoje predmete sa svim detaljima i sadržajem pismena možete vidjeti eKomunikacija https://usluge.pravosudje.hr/komun